In [356]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import PyPDF2
import os
from datetime import datetime
from time import sleep
year = 2019
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.160 Safari/537.36'}


while year >= 2010:
    url = 'https://www.supremecourt.gov/oral_arguments/argument_transcript/'+str(year)
    r = requests.get(url,headers=headers)
    print(r.status_code)
    soup = BeautifulSoup(r.text,"html.parser")

    table_rows = soup.find_all("tr")
    fails = []
    first = True
    links = []
    for row in table_rows:
        link = row.find("a", href=True)
        if link is not None:
            if link["href"].startswith("../"):
                links.append(link)
    for link in links:
        getPDF(link,year)
    year -=1
    sleep(10)

200
200
200
200
200
200
200
200


ValueError: substring not found

In [373]:
fails =[]
success = []
for pdf in os.listdir("pdfs"):
    try:
        if pdf.startswith('17') or pdf.startswith('18') or pdf.startswith('19') or pdf.startswith('20') or pdf.startswith('21') or re.search('\d\d\d-',pdf) is not None:
            caseNum  = pdf.replace(".pdf","")
            
            file = "pdfs\\"+pdf
            print(file)
            try:
                case = PyPDF2.PdfFileReader(file)
                meta_data = getMetaData(case,caseNum)
                case_data = parseArgument(case,meta_data)
            except Exception as e:
                print(e)
            if len(case_data) !=0 :
                data = pd.DataFrame(case_data,columns=["speaker","quote","case_name","case_number","petitioner","respondent","date_argued","et_al","official_page_count","actual_page_count"])
                if first:
                    df = data
                    first = False
                else:
                    df = pd.concat([df,data],axis=0)
                success.append(caseNum)
            else:
                fails.append(caseNum)
        df.to_csv("supreme_court_oral_args.csv",index=False)
    except Exception as e:
        print(e)
with open("fails.txt","wb") as f:
    f.write(str(fails))
f.close()

with open("success.txt","wb") as f:
    f.write(str(success))
f.close()


pdfs\141-orig.pdf
187
pdfs\142-orig.pdf
274
pdfs\143-orig.pdf
249
pdfs\17-1011.pdf
132
pdfs\17-1026.pdf
233
pdfs\17-1042.pdf
130
pdfs\17-1077.pdf
187
pdfs\17-1091.pdf
194
pdfs\17-1094.pdf
221
pdfs\17-1104.pdf
225
pdfs\17-1107.pdf
293
pdfs\17-1174.pdf
202
pdfs\17-1184.pdf
223
pdfs\17-1201.pdf
247
pdfs\17-1229.pdf
153
pdfs\17-1268.pdf
239
pdfs\17-1272.pdf
260
pdfs\17-1299.pdf
160
pdfs\17-130.pdf
206
pdfs\17-1307.pdf
213
pdfs\17-1471.pdf
261
pdfs\17-1484.pdf
unsupported operand type(s) for -: 'NoneType' and 'int'
pdfs\17-1498.pdf
unsupported operand type(s) for -: 'NoneType' and 'int'
pdfs\17-155.pdf
160
pdfs\17-1594.pdf
146
pdfs\17-1606.pdf
190
pdfs\17-1618.pdf
250
pdfs\17-1625.pdf
unsupported operand type(s) for -: 'NoneType' and 'int'
pdfs\17-1657.pdf
268
pdfs\17-1672.pdf
238
pdfs\17-1678.pdf
unsupported operand type(s) for -: 'NoneType' and 'int'
pdfs\17-1702.pdf
320
pdfs\17-1705.pdf
370
pdfs\17-1712.pdf
272
pdfs\17-1717.pdf
372
pdfs\17-2.pdf
224
pdfs\17-204.pdf
202
pdfs\17-21.pdf
158

TypeError: a bytes-like object is required, not 'str'

In [361]:

def getMetaData(case, case_number):
    first_page = case.getPage(0)

    text = first_page.extractText().replace("-","").replace(".","").replace(")","").replace(",","").strip().split("\n")
    text_cleaned = []
    for item in text:
        text_cleaned.append(item.strip())

    try:
        pindex = text_cleaned.index("Petitioners")-1
    except ValueError:
        try:
            pindex = text_cleaned.index("Petitioner")-1
        except ValueError:
            try: 
                pindex = text_cleaned.index("Plaintiff")
            except:
                pindex = None
        
    try:
        rindex = text_cleaned.index("Respondents")-1
    except ValueError:
        try:
            rindex = text_cleaned.index("Respondent")-1
        except ValueError:
            try: 
                rindex = text_cleaned.index("Defendants")
            except:
                rindex = None
    if pindex is not None or rindex is not None:
        x = 0
        cont = True
        petitioner = ""
        while cont:
            if "SUPREME COURT OF" in text_cleaned[pindex-x]:
                cont = False
            else:
                petitioner = text_cleaned[pindex-x] + " " + petitioner
                x+=1
            if x > 5: 
                cont = False

        cont = True
        x = 0
        respondent = ""
        while cont:
            if text_cleaned[rindex-x].startswith("v "):
                cont = False
            else:
                respondent = text_cleaned[rindex-x] + " " + respondent
                x+=1
            if x > 5: 
                cont = False
        respondent = re.sub("(\d)(\d)?(\d)?(\d)?(\d)? ","",respondent)
    else:
        petitioner = ""
        respondent = ""


    if "ET AL" in respondent:
        et_al = True
        respondent = respondent.replace(" ET AL","")
    if "ET AL" in petitioner:
        et_al = True
        petitioner = petitioner.replace(" ET AL","")
    else: 
        et_al = False
    for item in text_cleaned:
        pages = None
        numPages = None
        if item.startswith("Pages:"):
            pages = item
            try:
                numPages = int(pages[pages.index("through ")+8:])
            except:
                pass
        date = None
        if item.startswith("Date:"):
            date = item.replace("Date: ","")
    
    return {
        "case_name":petitioner + " V. "+ respondent,
        "case_number": case_number,
        "petitioner":petitioner,
        "respondent":respondent,
        "date_argued":date,
        "et_al": et_al,
        "official_page_count":numPages,
        "actual_page_count":case.getNumPages()
    }


In [309]:

def parseArgument(case,meta_data):
    speakerLines =[]
    speaker = None
    for i in range(case.getNumPages()):
        page = case.getPage(i)
        text = page.extractText()
        lines = text.strip().split("\n")
        remove = []
        for line in lines:
            
            if line.find("ORAL ARGUMENT OF:") != -1:
                appearance_page = page
            if line.replace(" ","").replace("Official","").isnumeric():
                remove.append(line)
            if line.find("-Subject to Final Review") != -1:
                remove.append(line)
        for line in remove:
            lines.pop(lines.index(line))
        current_line=""
        for line in lines:
            new_speaker = getSpeaker(speaker,line)
            if new_speaker == speaker:
                current_line += removeText(line)
            else:
                if speaker is not None:
                    current_line = current_line.replace(speaker+": ","")
                    speakerLines.append([speaker,current_line,meta_data["case_name"],meta_data["case_number"],meta_data["petitioner"],meta_data["respondent"],meta_data["date_argued"],meta_data["et_al"],meta_data["official_page_count"],meta_data["actual_page_count"]])
                current_line = removeText(line)
                speaker = new_speaker
    print(len(speakerLines))
    return speakerLines

In [272]:
def getSpeaker(speaker, line):
    try:
        if line.find("MR.")!= -1:
            speaker = line[:line.index(":")].replace("-","")
        if line.find("MS.")!= -1:
            speaker = line[:line.index(":")].replace("-","")
        if line.find("JUSTICE") != -1:
            speaker = line[:line.index(":")].replace("-","")
    except:
        pass
    return speaker


In [347]:
def getPDF(link,year):
    transcript = link["href"]
    url = transcript.replace("../","https://www.supremecourt.gov/oral_arguments/")
    transcript = transcript.replace("../argument_transcripts/"+str(year)+"/","")
    caseNum  = transcript[:transcript.index("_")]
    r = requests.get(url, stream=True)
    pdf = open("C:\\Users\\jlabado1\\Documents\\UH\\Data Requests\\Melissa - 5-10\\pdfs\\"+caseNum+".pdf","wb")
    pdf.write(r.content)
    pdf.close()
    return pdf
    
    

In [318]:
import re 

def removeText(text):
    cleaned_text = re.sub("\d\d( )?( )?(--)?\d( )?Official","",text)
    cleaned_text = re.sub("--"," --",cleaned_text)
    return cleaned_text

In [342]:
first_page = case.getPage(0)

case = PyPDF2.PdfFileReader("pdfs\\20-480.pdf")
case_number = "20-1114"
text = first_page.extractText().replace("-","").replace(".","").replace(")","").replace(",","").strip().split("\n")
text_cleaned = []
for item in text:
    text_cleaned.append(item.strip())

try:
    pindex = text_cleaned.index("Petitioners")-1
except ValueError:
    try:
        pindex = text_cleaned.index("Petitioner")-1
    except ValueError:
        try: 
            pindex = text_cleaned.index("Plaintiff")
        except:
            pindex = None
    
try:
    rindex = text_cleaned.index("Respondents")-1
except ValueError:
    try:
        rindex = text_cleaned.index("Respondent")-1
    except ValueError:
        try: 
            rindex = text_cleaned.index("Defendants")
        except:
            rindex = None
if pindex is not None or rindex is not None:
    x = 0
    cont = True
    petitioner = ""
    while cont:
        if "SUPREME COURT OF" in text_cleaned[pindex-x]:
            cont = False
        else:
            petitioner = text_cleaned[pindex-x] + " " + petitioner
            x+=1
        if x > 5: 
            cont = False

    cont = True
    x = 0
    respondent = ""
    while cont:
        if text_cleaned[rindex-x].startswith("v "):
            cont = False
        else:
            respondent = text_cleaned[rindex-x] + " " + respondent
            x+=1
        if x > 5: 
            cont = False
    respondent = re.sub("(\d)(\d)?(\d)? ","",respondent)
else:
    petitioner = ""
    respondent = ""


if "ET AL" in respondent:
    et_al = True
    respondent = respondent.replace(" ET AL","")
if "ET AL" in petitioner:
    et_al = True
    petitioner = petitioner.replace(" ET AL","")
else: 
    et_al = False
for item in text_cleaned:
    pages = None
    numPages = None
    if item.startswith("Pages:"):
        pages = item
        numPages = int(pages[pages.index("through ")+8:])
    date = None
    if item.startswith("Date:"):
        date = item.replace("Date: ","")
        date = datetime.strptime(date,"%B %d %Y")

data_check= {
    "case_name":petitioner + " V. "+ respondent,
    "case_number": case_number,
    "petitioner":petitioner,
    "respondent":respondent,
    "date_argued":date,
    "et_al": et_al,
    "official_page_count":numPages,
    "actual_page_count":case.getNumPages()
}

data_check

{'case_name': 'DAVID BRYON BABCOCK  V. KILOLO KIJAKAZI ACTING COMMISSIONER OF SOCIAL SECURITY ',
 'case_number': '20-1114',
 'petitioner': 'DAVID BRYON BABCOCK ',
 'respondent': 'KILOLO KIJAKAZI ACTING COMMISSIONER OF SOCIAL SECURITY ',
 'date_argued': None,
 'et_al': False,
 'official_page_count': None,
 'actual_page_count': 68}

In [375]:
print(len(fails)/len(success))

0.06153846153846154
